In [1]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
import kaggle
import kagglehub
import json
import zipfile

In [2]:
# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Prepare the dateset

"""
@credits: 
 - Dataset provided by Kaggle: https://www.kaggle.com/datasets/manideep1108/tusimple
"""

# Download the dataset
!kaggle datasets download -d manideep1108/tusimple 

# Extract the dataset
dataset_zip = "tusimple.zip"
extract_folder = "tusimple_dataset"

os.makedirs(extract_folder, exist_ok=True)
with zipfile.ZipFile(dataset_zip, "r") as zip_ref:
    zip_ref.extractall(extract_folder)

print("Dataset downloaded and extracted successfully.")

# Define dataset paths
dataset_dir = "tusimple_dataset"
img_dir = os.path.join(dataset_dir, "TUSimple/train_set/clips")
json_file = os.path.join(dataset_dir, "TUSimple/train_set/label_data_0313.json")

# Check if JSON file exists
if not os.path.exists(json_file):
    raise FileNotFoundError(f"Error: {json_file} not found! Ensure the dataset is correctly downloaded.")


Dataset URL: https://www.kaggle.com/datasets/manideep1108/tusimple
License(s): copyright-authors
Dataset downloaded and extracted successfully.


In [72]:
# Load TuSimple Dataset and Label Images


"""
@credits:
 - PyTorch DataLoader and Dataset structure inspired by official PyTorch documentation: 
   https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
 - JSON handling and best practices for dataset preprocessing based on real-world machine learning workflows:  
   * Kaggle discussion on data preprocessing: https://www.kaggle.com/discussions/general/222537  
   * Scikit-Learn preprocessing guide: https://scikit-learn.org/stable/modules/preprocessing.html  
"""

class LaneDataset(Dataset):
    def __init__(self, img_dir, labels, transform=None):
        self.img_dir = img_dir
        self.labels = {f: labels[f] for f in labels if os.path.exists(os.path.join(img_dir, f))}  # Ensure valid images only
        self.image_files = list(self.labels.keys())
        self.transform = transform

        if len(self.image_files) == 0:
            raise ValueError("No valid images found! Check dataset paths.")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)

        image = cv2.imread(img_path)
        if image is None:
            print(f"WARNING: Missing image -> {img_path}")
            return self.__getitem__((idx + 1) % len(self))  # Skip missing image

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[img_name]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)


# Load TuSimple Dataset
dataset_dir = "./tusimple_dataset/TUSimple/train_set"
json_files = ["label_data_0313.json", "label_data_0531.json", "label_data_0601.json"]

train_labels = {}
for json_file in json_files:
    json_path = os.path.join(dataset_dir, json_file)
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            data = [json.loads(line) for line in f]  # JSONL format
            for item in data:
                raw_file = item["raw_file"]  # Example: "clips/0313-1/17260/20.jpg"
                if os.path.exists(os.path.join(dataset_dir, raw_file)):  # ✅ Only add existing files
                    train_labels[raw_file] = 1 if len(item["lanes"]) > 0 else 0
    else:
        print(f"Warning: {json_path} not found!")

# Correct image directory (train_set is root)
img_dir = dataset_dir  

# Load dataset safely
try:
    train_dataset = LaneDataset(img_dir, train_labels, transform=None)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    print(f"Total images in dataset: {len(train_dataset)}")
except ValueError as e:
    print(f"Dataset error: {e}")


Total images in dataset: 3626


In [73]:
# Read TuSimple JSON labels

"""
@credits:
 - Kaggle discussion on data preprocessing: https://www.kaggle.com/discussions/general/222537  
 - Scikit-Learn preprocessing guide: https://scikit-learn.org/stable/modules/preprocessing.html  
"""

json_file = './tusimple_dataset/TUSimple/train_set/label_data_0313.json'

label_dict = {}
with open(json_file, 'r') as f:
    data = [json.loads(line) for line in f]
for item in data:
    filename = item["raw_file"]
    if len(item["lanes"]) > 0:
        label_dict[filename] = 1  # Lane Present
    else:
        label_dict[filename] = 0  # No Lane


In [74]:
# Split dataset

"""
@credits:
 - Data splitting methodology inspired by Scikit-Learn documentation:  
   * Train-test split: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html  
 - PyTorch DataLoader and Dataset structure based on PyTorch documentation:  
   * DataLoader tutorial: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html  
"""

# Split dataset into train, validation, and test sets
train_files, test_files = train_test_split(list(label_dict.keys()), test_size=0.3, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.3, random_state=42)  # 30% of train as validation

train_labels = {k: label_dict[k] for k in train_files}
val_labels = {k: label_dict[k] for k in val_files}
test_labels = {k: label_dict[k] for k in test_files}

# Create train, validation, and test datasets
train_dataset = LaneDataset(img_dir, train_labels, transform)
val_dataset = LaneDataset(img_dir, val_labels, transform)
test_dataset = LaneDataset(img_dir, test_labels, transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)  # Validation loader
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [75]:
# Applying Data Transformations

"""
@credits:
 - Data augmentation techniques from PyTorch documentation:  
   * Transformations: https://pytorch.org/vision/stable/transforms.html  
   * Data augmentation best practices: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html  
"""

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.3),  
    transforms.RandomRotation(5),  
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05), 
    transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)), 
    transforms.ToTensor(),
])


In [76]:
"""
@credits:
 - PyTorch DataLoader and Dataset structure from official PyTorch documentation:  
   * DataLoader tutorial: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html  
"""


# Define correct dataset paths
dataset_dir = "./tusimple_dataset/TUSimple/train_set"
json_files = [
    "label_data_0313.json",
    "label_data_0531.json",
    "label_data_0601.json",
]

# Load and merge label data from multiple JSON files
train_labels = {}
for json_file in json_files:
    json_path = os.path.join(dataset_dir, json_file)
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            data = [json.loads(line) for line in f]  # JSONL format (line-separated JSON)
            for item in data:
                train_labels[item["raw_file"]] = 1 if len(item["lanes"]) > 0 else 0
    else:
        print(f"Warning: {json_path} not found!")

# Define correct image directory
img_dir = dataset_dir

# Load dataset
train_dataset = LaneDataset(img_dir, train_labels, transform)
test_dataset = LaneDataset(img_dir, test_labels, transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Debug output
print(f"Total images in dataset: {len(train_dataset)}")


Total images in dataset: 3626


In [77]:
# Defining the CNN Model

"""
@credits:
 - CNN architecture from PyTorch official ResNet example:  
   * ResNet model from PyTorch documentation: https://pytorch.org/hub/pytorch_vision_resnet/  
 - Transfer learning and fine-tuning best practices based on PyTorch documentation:  
   * Transfer learning tutorial: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html  
"""

class LaneCNN(nn.Module):
    def __init__(self):
        super(LaneCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False  # Freeze all layers except the final layer
            
        self.model.fc = nn.Sequential(
            nn.Dropout(0.7),
            nn.Linear(self.model.fc.in_features, 2)
        )

    def forward(self, x):
        return self.model(x)


In [78]:
# Training the Model

"""
@credits:
 - CNN architecture from PyTorch official ResNet example:  
   * ResNet model from PyTorch documentation: https://pytorch.org/hub/pytorch_vision_resnet/  
 - Transfer learning and fine-tuning best practices based on PyTorch documentation:  
   * Transfer learning tutorial: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html  
 - PyTorch training loop and optimization from PyTorch tutorials:  
   * Training a model tutorial: https://pytorch.org/tutorials/beginner/nn_tutorial.html  
   * Learning rate scheduler: https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.StepLR  
"""

# Initialize Model, Loss, and Optimizer
model = LaneCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# Train Model
num_epochs = 20
print("Starting Training...")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Print batch progress every 10 batches
        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Validation Step (After Each Epoch)
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)


    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")

print("Training Finished!")


Starting Training...
Epoch [1/20], Batch [0/227], Loss: 1.4868
Epoch [1/20], Batch [10/227], Loss: 0.0193
Epoch [1/20], Batch [20/227], Loss: 0.0008
Epoch [1/20], Batch [30/227], Loss: 0.0002
Epoch [1/20], Batch [40/227], Loss: 0.0010
Epoch [1/20], Batch [50/227], Loss: 0.0003
Epoch [1/20], Batch [60/227], Loss: 0.0005
Epoch [1/20], Batch [70/227], Loss: 0.0007
Epoch [1/20], Batch [80/227], Loss: 0.0014
Epoch [1/20], Batch [90/227], Loss: 0.0001
Epoch [1/20], Batch [100/227], Loss: 0.0003
Epoch [1/20], Batch [110/227], Loss: 0.0003
Epoch [1/20], Batch [120/227], Loss: 0.0002
Epoch [1/20], Batch [130/227], Loss: 0.0003
Epoch [1/20], Batch [140/227], Loss: 0.0009
Epoch [1/20], Batch [150/227], Loss: 0.0001
Epoch [1/20], Batch [160/227], Loss: 0.0016
Epoch [1/20], Batch [170/227], Loss: 0.0002
Epoch [1/20], Batch [180/227], Loss: 0.0005
Epoch [1/20], Batch [190/227], Loss: 0.0002
Epoch [1/20], Batch [200/227], Loss: 0.0005
Epoch [1/20], Batch [210/227], Loss: 0.0001
Epoch [1/20], Batch [2

In [79]:
# Saving the Trained Model

"""
@credits:
 - PyTorch saving and loading models based on PyTorch documentation:  
   * Saving models: https://pytorch.org/tutorials/beginner/saving_loading_models.html  
"""

torch.save(model.state_dict(), "lane_cnn_classifier.pth")


In [80]:
# Evaluating the Model

"""
@credits:
 - PyTorch evaluation loop inspired by PyTorch tutorials:  
   * Model evaluation: https://pytorch.org/tutorials/beginner/saving_loading_models.html#evaluation  
"""

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct/total:.4f}")


Test Accuracy: 1.0000
